In [125]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

In [126]:
df = pd.read_csv("full2.csv")
year = pd.read_csv("2018.csv")

In [127]:
power_5 = ["Big Ten", "ACC", "SEC", "Pac-10", "Big 12", "Pac-12", "WAC"]
group_5 = ["CUSA", "MAC", "MWC", "American", "Sun Belt"]
fcs = ["Big East", "Unknown", "Ind", "MVC"]
df.loc[df.Conference.isin(power_5), "Conference"] = "Power 5"
df.loc[df.Conference.isin(group_5), "Conference"] = "Group 5"
df.loc[df.Conference.isin(fcs), "Conference"] = "FCS"

In [128]:
df = df.drop(["College", "Name", "Pick"], axis=1)

In [129]:
df = pd.get_dummies(df)

In [130]:
train = df[df.Year != 2018]
test = df[df.Year == 2018]

In [131]:
test = test.drop("Year", axis=1)
train = train.drop("Year", axis=1)

In [132]:
train_X = train.drop("Round", axis=1)
test_X = test.drop("Round", axis=1)
train_y = train.Round
train = xgb.DMatrix(train_X, label=train_y)
test = xgb.DMatrix(test_X, label=test_y)

In [133]:
#model = XGBRegressor()
model = LinearRegression()

In [134]:
model.fit(train_X, train_y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [135]:
ranks = np.argsort(list(model.predict(test_X))) + 1

In [138]:
ranks

array([132,  19, 195,  20, 262,  95, 151,  77,  49, 255, 209,   1, 240,
        90,  50, 156,  88, 241,  76, 175,  65, 135,  10, 207, 163,   8,
         5, 129, 220,  62,  47, 198,  78, 210, 103, 154, 136, 159,  16,
       124,  40, 178, 230, 176,  68, 282,  97, 100, 123,  84, 203,  69,
       260, 173, 166, 181,  66, 200, 118,  11, 110, 171, 105, 223, 244,
       246,  35, 130,  70, 275, 183,  79, 226, 126, 199, 269, 184, 236,
       194,  21,  28, 190,  44, 141, 218,   2,   3, 174, 116,  41, 119,
       111, 167, 264,  64, 169, 145,  93, 182, 150,  39, 109, 142, 285,
       205, 160, 215, 271, 239,  17, 234, 193,  53, 152, 186,  33,  80,
       261,  13, 187, 263, 221, 242,  36, 212, 158, 107, 214, 128, 180,
       231, 252, 249, 228, 157, 120, 238, 106,  34, 192, 248,  54, 155,
        58, 125, 138, 258, 277,  15, 201,  89, 131, 114, 219,  25, 101,
        86, 204, 253, 161, 137,  81, 146, 202,  30,  56, 188, 143, 273,
       225,  96,  18, 245, 280,  37, 227, 272,  83,  57,  91,  9

In [139]:
params = {'colsample_bytree': 0.7,
 'eta': 0.05,
 'eval_metric': 'mae',
 'max_depth': 6,
 'min_child_weight': 15,
 'objective': 'reg:linear',
 'subsample': 0.7}

In [146]:
num_boost_round = 999
model = xgb.train(
    params,
    train,
    num_boost_round=num_boost_round,
    #evals=[(test, "Test")],
    #early_stopping_rounds=10
)

In [141]:
pred1 = model.predict(test)

In [142]:
lm = LinearRegression()
lm.fit(train_X, train_y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [143]:
pred2 = lm.predict(test_X)

In [155]:
pred3 = 0.6 * pred1 + 0.4 * pred2

In [156]:
np.argsort(pred3) + 1

array([ 19, 132,  77, 118, 176, 178, 199, 255, 154, 175,  49,  47, 241,
       187, 207, 210,   8, 136,  65, 129, 135, 160, 240, 262, 230, 195,
        50,  10, 156, 159, 244,  79, 119,   5,  76, 171, 158, 215, 276,
       145,  62, 173, 101, 123,  88, 209, 181, 126, 260, 220,  78, 282,
         1, 151, 214,  40, 163,  68, 167,  28, 226, 141, 228, 194,  66,
       157, 208,  95,  84,  69, 103,  15,  80, 205, 231,  70,  90, 124,
       203,  11, 223,  44, 285, 225, 130,  61, 271, 185, 109,   2,  45,
       169,  52, 166,  20,  36,  17, 100, 105, 200, 110,  41,   9, 183,
       263, 128, 192,  34,   3, 146, 150, 184, 170,  37, 152,  86,  35,
        16,  13,  39, 174, 248, 111, 182,  97,  93, 186, 236, 277, 140,
        54,  67, 221, 261, 275, 239, 243, 116, 155, 246, 206,  89, 143,
       138, 180, 247, 164,  58, 218, 259, 131,  63,  99,  94, 104, 216,
       142,  25,  33,  53,  30,  21, 273, 249, 253, 264,  75,  56, 137,
        64, 193, 188, 162,  51, 272, 258, 252, 269, 198, 190, 26

In [159]:
np.argsort(pred1) +1

array([ 19, 118, 132, 176, 178, 199,  77, 187, 154, 276,  47, 175, 160,
       255, 210, 136, 207, 241,  49,   8, 129, 135, 158,  65, 101, 230,
       119, 208, 215, 244,  79, 145, 159, 240, 171,  10, 214, 228, 173,
         5, 185, 157, 126,  50,  15, 156, 123, 167,  61,  76, 181,  62,
       260, 140,  28, 141, 226, 262, 194, 282, 225, 231,  80,  45,  68,
         9,  40,  78, 220, 205,  52,  66,  88, 163,  69,  84, 285, 271,
        70, 209, 195,  44, 109, 170, 103,   1,  36, 223, 203, 146,  11,
       130,   2,  17,  37, 192, 169, 124, 263, 128,  34, 206, 151,  86,
        67, 164,  41, 248, 243, 152, 166, 150,  94, 105, 110, 183, 200,
         3,  13, 100,  95,  90, 277, 184, 259,  54,  39, 186, 174, 221,
       111, 182, 143, 261,  89, 247,  35, 155,  93, 104, 216, 239,  63,
       138, 102,  99, 235, 236, 180, 162,  16, 116, 131, 265,  14,  58,
       275,  97,  51,  75, 246,  30, 273,  25, 147,  56, 253,  85, 218,
       137, 188, 272,  20,  57,  33, 249,  53, 270, 142, 139, 22

In [160]:
for row in np.argsort(pred1):
    print(year.at[row, "Name"])

Saquon Barkley
B.J. Hill
Lamar Jackson
Kolton Miller
D.J. Moore
Harrison Phillips
Tremaine Edmunds
Joseph Noteboom
Harold Landry
Eddy Wilson
D.J. Chark
Anthony Miller
Chase Litton
Vita Vea
Justin Reid
Malik Jefferson
Frank Ragnow
Josh Sweat
Bradley Chubb
Josh Allen
J.C. Jackson
Derwin James
Rick Leonard
Sam Darnold
Colby Gossett
Braden Smith
Holton Hill
Martinas Rankin
Josh Rosen
Wyatt Teller
Deshon Elliott
Peter Kalambayi
Tyquan Lewis
Courtland Sutton
Tarvarus McFadden
Mark Andrews
Korey Robertson
Nic Shimonek
Quenton Meeks
Jaire Alexander
Derrick Nnadi
Tanner Lee
Godwin Igwebuike
Nick Chubb
Anthony Averett
Allen Lazard
Sam Hubbard
Baker Mayfield
Tyrell Crosby
Terrell Edmunds
Ryan Nall
Dane Cruikshank
Denzel Ward
Jamarco Jones
Quin Blanding
Justin Jones
Tim Settle
James Washington
Daron Payne
Isaiah Wynn
Deadrin Senat
Roquan Smith
Leighton Vander esch
Andre Chachere
Carlton Davis
Marcus Allen
Antonio Callaway
Duke Ejiofor
Tegray Scales
Scott Quessenberry
Will Clapp
Marcus Davenport
Mi